In [1]:
# Imports
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
import shutil

In [2]:
# === STEP 1: GENERATE DATA FROM 2 PHOTOS ===
# This script takes friend1.jpg and friend2.jpg and creates 200 variations of each.

# Configuration
INPUT_FILES = {
    "Friend_1": "friend1.jpg", 
    "Friend_2": "friend2.jpg"
}
SAMPLES_PER_PERSON = 200 

def generate_augmented_data():
    # Define how to "fake" the new images
    datagen = ImageDataGenerator(
        rotation_range=20,      
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        shear_range=0.1,        
        zoom_range=0.1,         
        horizontal_flip=True,   
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )

    base_dir = 'dataset/train'
    
    # Clean up old dataset if exists to avoid mixing
    if os.path.exists('dataset'):
        shutil.rmtree('dataset')
    
    print("Starting Data Generation...")
    
    for class_name, file_name in INPUT_FILES.items():
        if not os.path.exists(file_name):
            print(f"⚠ ERROR: Could not find '{file_name}'. Please upload it first!")
            continue
            
        save_dir = os.path.join(base_dir, class_name)
        os.makedirs(save_dir, exist_ok=True)
        
        try:
            img = load_img(file_name) 
            x = img_to_array(img)     
            x = x.reshape((1,) + x.shape) 
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            continue

        i = 0
        print(f"Generating {SAMPLES_PER_PERSON} images for {class_name}...")
        for batch in datagen.flow(x, batch_size=1, save_to_dir=save_dir, 
                                save_prefix=class_name, save_format='jpg'):
            i += 1
            if i >= SAMPLES_PER_PERSON:
                break
                
    print("\n✓ Dataset Created! Ready for training.")

generate_augmented_data()

Starting Data Generation...
Generating 200 images for Friend_1...
Generating 200 images for Friend_2...

✓ Dataset Created! Ready for training.


In [3]:
# === STEP 2: LOAD DATA ===
base_dir = 'dataset/train'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 # 20% for validation
)

print("Loading Training Data...")
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

print("Loading Validation Data...")
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Loading Training Data...
Found 320 images belonging to 2 classes.
Loading Validation Data...
Found 79 images belonging to 2 classes.


In [4]:
# === STEP 3: BUILD MODEL (FOR 2 CLASSES) ===
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    
    # Output Layer: 2 neurons for 2 friends
    layers.Dense(2, activation='softmax') 
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
# === STEP 4: TRAIN ===
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=15, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)

Epoch 1/15
10/10 [==============================] - 36s 3s/step - loss: 0.7135 - accuracy: 0.7781 - val_loss: 0.1376 - val_accuracy: 1.0000
Epoch 2/15
10/10 [==============================] - 28s 3s/step - loss: 0.0304 - accuracy: 1.0000 - val_loss: 1.4260e-05 - val_accuracy: 1.0000
Epoch 3/15
10/10 [==============================] - 29s 3s/step - loss: 1.8399e-06 - accuracy: 1.0000 - val_loss: 8.5495e-07 - val_accuracy: 1.0000
Epoch 4/15
10/10 [==============================] - 31s 3s/step - loss: 7.6405e-07 - accuracy: 1.0000 - val_loss: 7.7742e-06 - val_accuracy: 1.0000
Epoch 5/15
10/10 [==============================] - 30s 3s/step - loss: 8.2365e-07 - accuracy: 1.0000 - val_loss: 1.6354e-06 - val_accuracy: 1.0000
Epoch 6/15
10/10 [==============================] - 29s 3s/step - loss: 1.4529e-07 - accuracy: 1.0000 - val_loss: 3.8557e-07 - val_accuracy: 1.0000
Epoch 7/15
10/10 [==============================] - 30s 3s/step - loss: 4.0233e-08 - accuracy: 1.0000 - val_loss: 5.5879e-08

In [6]:
# === STEP 5: SAVE ===
os.makedirs('models', exist_ok=True)
model.save('models/face_recognition_model.h5')
print("Model saved as 'models/face_recognition_model.h5'")

# Verify classes
print("Class Mapping:", train_generator.class_indices)

Model saved as 'models/face_recognition_model.h5'
Class Mapping: {'Friend_1': 0, 'Friend_2': 1}
